# Calculating Daily PNA Values.

Calculating daily values for the PNA using data from: https://psl.noaa.gov/thredds/catalog/Datasets/20thC_ReanV3/Dailies/catalog.html

In [1]:
#Import modules needed.
import xarray as xr
import datetime as dt

In [2]:
#Read in files (500 mb heights, mean 500 mb heights, stdev 500 mb heights) using xarray and select 500 mb for the heights.
hgt = xr.open_dataset("/home/scratch/20CR_v3/daily_hgt_1836_2015.nc")
z500 = hgt.sel(level = 500.)
z500_mean = xr.open_dataset("/home/scratch/20CR_v3/climos/z500_mean_dayofyear_1836_2015.nc")
z500_std = xr.open_dataset("/home/scratch/20CR_v3/climos/z500_stdev_dayofyear_1836_2015.nc")


## Calculate the PNA Index Value

The equation used is 

$$
PNA = 0.25 * (Z[20^{\circ}N, 160^{\circ}W] - Z[45^{\circ}N, 165^{\circ}W] + Z[55^{\circ}N, 115^{\circ}W] - Z[30^{\circ}N, 85^{\circ}W])
$$

with each component being standardized 500 geopotential height anomaly values.

In [3]:
#Use for loop to calculate values of the PNA.
#Create list variable.
PNA_index = []

#Begin for loop using times values in the heights file.
for i in z500.time.values:
    
    #Grab current time and then convert it to datetime that can be used to match the day of the year.
    z500_current = z500.sel(time = str(i))
    current_day = dt.datetime.strptime(str(z500_current.time.values)[0:10], '%Y-%m-%d')
    dayofyear = int(current_day.strftime('%j'))
    
    #Calculate the standized anomaly using the proper day of the year and the current day.
    z500_std_anom = (z500_current.hgt - z500_mean.hgt.sel(dayofyear = dayofyear)) / z500_std.hgt.sel(dayofyear = dayofyear)
    
    #Select the specfic points used to calculate the PNA.
    p1_std_anom = z500_std_anom.sel(lat = 20., lon = 160. + 180.)
    p2_std_anom = z500_std_anom.sel(lat = 45., lon = 165. + 180.)
    p3_std_anom = z500_std_anom.sel(lat = 55., lon = 115. + 180.)
    p4_std_anom = z500_std_anom.sel(lat = 30., lon = 85. + 180.)
    
    #Calculate the PNA value.
    PNA = 0.25*(p1_std_anom - p2_std_anom + p3_std_anom - p4_std_anom)
    
    #Append the value to a list.
    PNA_index.append(PNA)

In [4]:
#Concat the list of PNA values to easily convert to a netcdf file.
PNA_DA = xr.concat(PNA_index, dim='time')
PNA_DA.to_netcdf('PNA_daily_values_1836_2015.nc')
PNA_DA

<xarray.DataArray 'hgt' (example: 65744)>
array([ 0.02952422, -0.02388081, -0.08281727, ...,  0.3579579 ,
        0.60439658,  0.21741638])
Coordinates:
    time       (example) datetime64[ns] 1836-01-01 1836-01-02 ... 2015-12-31
    level      float32 500.0
    dayofyear  (example) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
Dimensions without coordinates: example